In [ ]:
import plotly.express as px
import pandas as pd

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('../../../open-grid-emissions/src/')

from filepaths import *
import validation

# Visualize data for a single BA

In [ ]:
ba = "MISO"
year = 2020


egrid_ba = pd.read_excel(
    f"{downloads_folder()}egrid/egrid{year}_data.xlsx",
    sheet_name=f"BA{str(year)[-2:]}",
    header=1,
    usecols=["BANAME", "BACODE", "BANOXRTO","BASO2RTA","BACO2RTA","BAC2ERTA"],
).rename(
    columns={
        "BANAME": "ba_name",
        "BACODE": "ba_code",
        "BANOXRTO": "generated_nox_rate_lb_per_mwh_for_electricity_adjusted",
        "BASO2RTA": "generated_so2_rate_lb_per_mwh_for_electricity_adjusted",
        "BACO2RTA": "generated_co2_rate_lb_per_mwh_for_electricity_adjusted",
        "BAC2ERTA":"generated_co2e_rate_lb_per_mwh_for_electricity_adjusted"
    }
)

hourly_consumed = pd.read_csv(f"{results_folder()}{year}/carbon_accounting/hourly/us_units/{ba}.csv", parse_dates=["datetime_local"]).set_index("datetime_local")
hourly_produced = pd.read_csv(f"{results_folder()}{year}/power_sector_data/hourly/us_units/{ba}.csv", parse_dates=["datetime_local"]).set_index("datetime_local")

In [ ]:
pollutant = "co2e"
rate_type = "for_electricity_adjusted"

egrid_value = egrid_ba.loc[egrid_ba["ba_code"] == ba, f"generated_{pollutant}_rate_lb_per_mwh_{rate_type}"].item()
data_to_graph = hourly_consumed[[f"consumed_{pollutant}_rate_lb_per_mwh_{rate_type}"]].rename(columns={f"consumed_{pollutant}_rate_lb_per_mwh_{rate_type}":"hourly_consumed"})
data_to_graph["month"] = data_to_graph.index.astype(str).str[:7]
data_to_graph["annual_consumed"] = data_to_graph[["hourly_consumed"]].mean().item()
monthly_consumed = data_to_graph.groupby("month")["hourly_consumed"].mean().reset_index().rename(columns={"hourly_consumed":"monthly_consumed"})
orig_index = data_to_graph.index
data_to_graph = data_to_graph.merge(monthly_consumed, how="left", on="month", validate="m:1").set_index(orig_index)
data_to_graph = data_to_graph.merge(hourly_produced.loc[hourly_produced["fuel_category"] == "total", f"generated_{pollutant}_rate_lb_per_mwh_{rate_type}"], how="inner", left_index=True, right_index=True).rename(columns={f"generated_{pollutant}_rate_lb_per_mwh_{rate_type}":"hourly_produced"})
data_to_graph["annual_produced"] = data_to_graph[["hourly_produced"]].mean().item()
data_to_graph["egrid_value"] = egrid_value
data_to_graph

In [ ]:
px.line(data_to_graph, x=data_to_graph.index, y=["hourly_produced","hourly_consumed","monthly_consumed","egrid_value","annual_produced","annual_consumed",], labels={"value":"lbCO2e/MWh"}, template="plotly_white", title=ba)

In [ ]:
px.line(data_to_graph, x=data_to_graph.index, y=["hourly_consumed","monthly_consumed","annual_consumed",], labels={"value":"lbCO2e/MWh"}, template="plotly_white", title=ba)

In [ ]:
def graph_hourly_data_by_fuel_category(hourly_data, ba, column_name, fuel_category_name, plot_type):
    fuel_color = {'natural_gas':'sienna',
              'coal':'black',
              'nuclear':'red',
              'biomass':'green',
              'geothermal':'orange',
              'wind':'blue',
              'solar':'gold',
              'petroleum':'purple',
              'hydro':'skyblue',
              'other':'lightgrey',
              'waste':'pink'}

    fuel_order = ['nuclear','geothermal','hydro','other','coal','biomass','petroleum','waste','solar','wind','natural_gas']

    data_to_graph = hourly_data[hourly_data['ba_code'] == ba]
    #data_to_graph = data_to_graph.groupby(['datetime_utc','fuel_category','data_source']).sum().reset_index()

    if plot_type == 'area':
        plot = px.area(data_to_graph, 
                x='datetime_utc', 
                y=column_name, 
                color=fuel_category_name, 
                color_discrete_map=fuel_color, 
                facet_col='data_source', 
                template='plotly_white',
                title=f'Hourly {column_name} data for {ba} by fuel type',
                category_orders={fuel_category_name:fuel_order}).update_traces(line={'width':0})
    elif plot_type == 'line':
        plot = px.line(data_to_graph, 
                x='datetime_utc', 
                y=column_name, 
                color=fuel_category_name, 
                color_discrete_map=fuel_color, 
                facet_col='data_source', 
                template='plotly_white',
                title=f'Hourly {column_name} data for {ba} by fuel type',
                category_orders={fuel_category_name:fuel_order}).update_traces(line={'width':0})
    
    return plot
        

In [ ]:
ba = "CISO"

graph_hourly_data_by_fuel_category(
    hourly_data=combined_data,
    ba=ba,
    column_name="net_generation_mwh",
    fuel_category_name="fuel_category",
    plot_type="area",
)


In [ ]:
graph_hourly_data_by_fuel_category(
    hourly_data=combined_data,
    ba=ba,
    column_name="co2_mass_lb",
    fuel_category_name="fuel_category",
    plot_type="area",
)
